In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
# moving average
ma_test_res = pd.read_pickle("ma_test_res.pkl")
all_trades = pd.read_pickle("all_trades.pkl")

In [3]:
all_trades.shape

(65771, 11)

In [4]:
ma_test_res.shape

(630, 8)

In [9]:
ma_test_res.head()

,pair,num_trades,total_gain,mashort,malong,CROSS
0,GBP_USD,564,128.2,4,8,MA_4_8
1,GBP_USD,309,-56.6,4,16,MA_4_16
2,GBP_USD,275,97.1,8,16,MA_8_16
3,GBP_USD,208,-475.6,4,32,MA_4_32
4,GBP_USD,156,-998.5,8,32,MA_8_32


In [6]:
ma_test_res = ma_test_res[['pair', 'num_trades', 'total_gain', 'mashort', 'malong']]

In [10]:
ma_test_res["CROSS"] = "MA_" + ma_test_res.mashort.map(str) + "_" + ma_test_res.malong.map(str)

/tmp/ipykernel_11829/86883899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ma_test_res["CROSS"] = "MA_" + ma_test_res.mashort.map(str) + "_" + ma_test_res.malong.map(str)


In [13]:
df_all_gains = ma_test_res.groupby(by=["CROSS", "mashort", "malong"], as_index=False).sum()

In [15]:
df_all_gains.sort_values(by="total_gain", ascending=False, inplace=True)

In [17]:
df_all_gains.head(10)

,CROSS,mashort,malong,num_trades,total_gain
27,MA_8_32,8,32,3162,2532.7
18,MA_4_64,4,64,2661,1524.5
8,MA_24_64,24,64,1424,1177.7
12,MA_32_64,32,64,1382,1100.8
25,MA_8_16,8,16,5783,1082.1
28,MA_8_64,8,64,1999,215.7
2,MA_16_32,16,32,2819,143.7
17,MA_4_32,4,32,4072,-59.0
3,MA_16_64,16,64,1633,-875.0
15,MA_4_16,4,16,6571,-1203.4


In [19]:
ma_8_16 = ma_test_res[ma_test_res.CROSS=="MA_8_16"].copy()

In [21]:
ma_8_16.sort_values(by="total_gain", ascending=False, inplace=True)

In [22]:
ma_8_16.head(10)

,pair,num_trades,total_gain,mashort,malong,CROSS
212,EUR_CAD,261,840.8,8,16,MA_8_16
512,NZD_CAD,252,673.8,8,16,MA_8_16
182,EUR_USD,284,491.6,8,16,MA_8_16
422,CAD_JPY,270,453.2,8,16,MA_8_16
602,CHF_JPY,280,358.0,8,16,MA_8_16
542,NZD_JPY,261,333.3,8,16,MA_8_16
362,USD_JPY,273,320.1,8,16,MA_8_16
332,USD_CAD,278,259.6,8,16,MA_8_16
482,NZD_USD,262,223.4,8,16,MA_8_16
392,USD_CHF,281,222.5,8,16,MA_8_16


In [23]:
total_pairs = len(ma_8_16.pair.unique())

In [25]:
ma_8_16[ma_8_16.total_gain>0].shape[0]

14

In [34]:
for cross in df_all_gains.CROSS.unique():
    df_temp = ma_test_res[ma_test_res.CROSS==cross]
    total_p = df_temp.shape[0]
    n_good = df_temp[df_temp.total_gain>0].shape[0]
    print(f"{cross:12} {n_good:4} {(n_good/total_p)*100:4.0f}%")

MA_8_32        16   76%
MA_4_64        12   57%
MA_24_64       13   62%
MA_32_64       12   57%
MA_8_16        14   67%
MA_8_64        10   48%
MA_16_32       11   52%
MA_4_32        11   52%
MA_16_64        8   38%
MA_4_16        12   57%
MA_4_96         8   38%
MA_4_8         10   48%
MA_16_96        8   38%
MA_8_96         8   38%
MA_32_96        9   43%
MA_24_96        7   33%
MA_24_32        8   38%
MA_4_128        8   38%
MA_64_128       5   24%
MA_32_128       8   38%
MA_8_128        5   24%
MA_24_128       8   38%
MA_64_96        5   24%
MA_16_128       5   24%
MA_16_256       4   19%
MA_8_256        5   24%
MA_24_256       5   24%
MA_4_256        2   10%
MA_32_256       6   29%
MA_64_256       6   29%


In [47]:
# crosses = df_all_gains.CROSS.unique()[:3]
crosses = df_all_gains.CROSS.unique()[[0,2,4]]


In [48]:
crosses

array(['MA_8_32', 'MA_24_64', 'MA_8_16'], dtype=object)

In [49]:
df_good = ma_test_res[(ma_test_res.CROSS.isin(crosses)) & (ma_test_res.total_gain>0)].copy()

In [53]:
df_good.head()

,pair,num_trades,total_gain,mashort,malong,CROSS
2,GBP_USD,275,97.1,8,16,MA_8_16
32,GBP_CAD,280,23.6,8,16,MA_8_16
40,GBP_CAD,69,87.6,24,64,MA_24_64
64,GBP_JPY,150,1099.0,8,32,MA_8_32
70,GBP_JPY,61,739.2,24,64,MA_24_64


In [58]:
# is a series, index to make it a list
# top pairs { df_good.pair.value_counts() }
our_pairs = list(df_good.pair.value_counts()[:7].index)

In [59]:
our_pairs

['CAD_JPY', 'NZD_JPY', 'EUR_USD', 'EUR_CAD', 'EUR_NZD', 'USD_JPY', 'USD_CHF']

In [62]:
all_trades.head(2)

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS
24,2021-01-25 09:00:00+00:00,1.36735,-0.000124,0.000810,-1,2.0,-2.0,GBP_USD,4,8,12.0,MA_4_8
36,2021-01-25 21:00:00+00:00,1.36755,0.000228,-0.000004,1,-11.3,-11.3,GBP_USD,4,8,6.0,MA_4_8


In [61]:
all_trades["CROSS"] = "MA_" + all_trades.MASHORT.map(str) + "_" + all_trades.MALONG.map(str)

In [72]:
trades_cad_jpy = all_trades[(all_trades.CROSS=="MA_8_32") & (all_trades.PAIR=="CAD_JPY")].copy()

In [73]:
trades_cad_jpy.head()

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS
53,2021-01-26 15:00:00+00:00,81.576,0.032219,-0.015875,1,-16.7,-16.7,CAD_JPY,8,32,21.0,MA_8_32
74,2021-01-27 12:00:00+00:00,81.409,-0.001375,0.018000,-1,8.4,-8.4,CAD_JPY,8,32,30.0,MA_8_32
104,2021-01-28 18:00:00+00:00,81.493,0.001406,-0.052625,1,-25.8,-25.8,CAD_JPY,8,32,10.0,MA_8_32
114,2021-01-29 04:00:00+00:00,81.235,-0.009031,0.004125,-1,36.6,-36.6,CAD_JPY,8,32,5.0,MA_8_32
119,2021-01-29 09:00:00+00:00,81.601,0.012344,-0.012000,1,24.1,24.1,CAD_JPY,8,32,80.0,MA_8_32


In [74]:
trades_cad_jpy['CUM_GAIN'] = trades_cad_jpy.GAIN.cumsum()


In [75]:
def plot_line(df_plot, name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot.CUM_GAIN,
        line=dict(width=2, color="#3d825f"),
        line_shape='spline',
        name=name,
        mode='lines'
    ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=15,r=15,b=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e",
        title=name)
    fig.update_xaxes(
        gridcolor="#3a4a54",
        showgrid=False,
        zeroline=False
    )
    fig.update_yaxes(
        linewidth=1,
        linecolor="#3a4a54",
        showgrid=False,
        zeroline=False
    )
    fig.show()

In [76]:
plot_line(trades_cad_jpy, "CAD_JPY")

In [77]:
c = 'MA_8_32'
for p in our_pairs:
    temp_df = all_trades[(all_trades.CROSS==c) & (all_trades.PAIR==p)].copy()
    temp_df['CUM_GAIN'] = temp_df.GAIN.cumsum()
    plot_line(temp_df, p + "_" + c)

In [78]:
for c in crosses:
    temp_df = all_trades[(all_trades.CROSS==c)].copy()
    temp_df = temp_df.groupby(by="time", as_index=False).sum()
    temp_df['CUM_GAIN'] = temp_df.GAIN.cumsum()
    plot_line(temp_df, c)